In [60]:
## mashup script for batch changes
import os, osgeo
# del os.environ['PROJ_LIB']
import math
from collections import Counter
from datetime import datetime, timedelta
from geopandas.tools import sjoin
from pyproj import proj
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import fiona
from fulcrum import Fulcrum
import rasterio
from rasterstats import zonal_stats
import xarray as xr
import numpy as np
from osgeo import gdal, gdalconst, osr
import sys, time, ogr
import libpysal
gdal.UseExceptions()  # not required, but a good idea
gdal.AllRegister()
# Allow division by zero
numpy.seterr(divide='ignore', invalid='ignore')

In [61]:
# Raster dataset
infile = r"D:\BPLA Dropbox\08 GIS-data\GIS-data-Riyadh\2022-02-NDVI-WVSatellite-image\2022 SateImage-NDVI-analysis-2048.tif"
outfile = r"D:\BPLA Dropbox\08 GIS-data\GIS-data-Riyadh\2022-02-NDVI-WVSatellite-image\2022 SateImage-NDVI-analysis-2048-bin-1.tif"
# datascr = rasterio.open(infile)
# out_profile = datascr.profile.copy()
# dst = rasterio.open(outfile, 'w', **out_profile)


In [ ]:
def zonal_stats(feat, input_zone_polygon, input_value_raster):

    # Open data
    raster = gdal.Open(input_value_raster)
    shp = osgeo.ogr.Open(input_zone_polygon)
    lyr = shp.GetLayer()

    # Get raster georeference info
    transform = raster.GetGeoTransform()
    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = transform[5]

    # Get extent of feat
    geom = feat.GetGeometryRef()
    if (geom.GetGeometryName() == 'MULTIPOLYGON'):
        count = 0
        pointsX = []; pointsY = []
        for polygon in geom:
            geomInner = geom.GetGeometryRef(count)    
            ring = geomInner.GetGeometryRef(0)
            numpoints = ring.GetPointCount()
            for p in range(numpoints):
                    lon, lat, z = ring.GetPoint(p)
                    pointsX.append(lon)
                    pointsY.append(lat)    
            count += 1
    elif (geom.GetGeometryName() == 'POLYGON'):
        ring = geom.GetGeometryRef(0)
        numpoints = ring.GetPointCount()
        pointsX = []; pointsY = []
        for p in range(numpoints):
                lon, lat, z = ring.GetPoint(p)
                pointsX.append(lon)
                pointsY.append(lat)

    else:
        sys.exit()

    xmin = min(pointsX)
    xmax = max(pointsX)
    ymin = min(pointsY)
    ymax = max(pointsY)

    # Specify offset and rows and columns to read
    xoff = int((xmin - xOrigin)/pixelWidth)
    yoff = int((yOrigin - ymax)/pixelWidth)
    xcount = int((xmax - xmin)/pixelWidth)+1
    ycount = int((ymax - ymin)/pixelWidth)+1

    # Create memory target raster
    target_ds = gdal.GetDriverByName('MEM').Create('', xcount, ycount, gdal.GDT_Byte)
    target_ds.SetGeoTransform((
        xmin, pixelWidth, 0,
        ymax, 0, pixelHeight,
    ))

    # Create for target raster the same projection as for the value raster
    raster_srs = osr.SpatialReference()
    raster_srs.ImportFromWkt(raster.GetProjectionRef())
    target_ds.SetProjection(raster_srs.ExportToWkt())

    # Rasterize zone polygon to raster
    gdal.RasterizeLayer(target_ds, [1], lyr, burn_values=[1])

    # Read raster as arrays
    banddataraster = raster.GetRasterBand(1)
    dataraster = banddataraster.ReadAsArray(xoff, yoff, xcount, ycount)#.astype(np.uint8)

    bandmask = target_ds.GetRasterBand(1)
    datamask = bandmask.ReadAsArray(0, 0, xcount, ycount)#.astype(np.uint8)

    # Mask zone of raster
    zoneraster = np.ma.masked_array(dataraster,  np.logical_not(datamask))

    # Calculate statistics of zonal raster


    # Cover = (np.count_nonzero(zoneraster)/np.count_nonzero(dataraster))
    # print (dataraster.shape)
    cover = (np.sum(zoneraster)/zoneraster.size)*100
    #).sum()/zoneraster.size*100))
    # print(cover)
    # print()
    # unique, counts = np.unique(zoneraster, return_counts=True)
    # print (unique, counts)
    # return (np.count_nonzero(zoneraster)/np.count_nonzero(dataraster))
    return cover
    
 
def loop_zonal_stats(input_zone_polygon, input_value_raster):

    shp = osgeo.ogr.Open(input_zone_polygon)
    lyr = shp.GetLayer()
    featList = range(lyr.GetFeatureCount())
    statDict = {}

    for FID in featList:
        feat = lyr.GetFeature(FID)
        meanValue = zonal_stats(feat, input_zone_polygon, input_value_raster)
        statDict[FID] = meanValue
    return statDict



In [ ]:
input_value_raster = r"D:\BPLA Dropbox\08 GIS-data\GIS-data-Riyadh\2022-02-NDVI-WVSatellite-image\2022 SateImage-NDVI-analysis-2048-bin-1.tif"

# Vector dataset(zones)
# input_zone_polygon = r"D:\BPLA Dropbox\03 Planning\1232-T2-GIS and Remote Sensing-TMO\02_GIS_Data\Training Data Riyadh\Loop_test.shp"
input_zone_polygon = r"C:\Users\DroneAdmin\Downloads\Urban-Environs-limit.shp"

os.environ['PROJ_LIB'] = r"C:\Users\DroneAdmin\Downloads\RS\venv\Lib\site-packages\rasterio\proj_data"
#os.environ['GDAL_DATA'] = r"C:\Users\DroneAdmin\Downloads\RS\venv\Lib\site-packages\osgeo\include\gdal"


rr = loop_zonal_stats(input_zone_polygon, input_value_raster)
print (rr)
#print (zonal_stats, input_zone_polygon, input_value_raster))

MemoryError: Unable to allocate 387. GiB for an array with shape (268283, 387160) and data type float32